In [10]:
import lightning as pl
import torch
from torch.utils.data import DataLoader, Dataset

# Define a simple dataset for demonstration purposes
class DummyDataset(Dataset):
    def __len__(self):
        return 100

    def __getitem__(self, idx):
        return torch.tensor([idx], dtype=torch.float), torch.tensor([idx], dtype=torch.float)

# Define the first child module
class ChildModule1(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.dummy_layer = torch.nn.Linear(1, 1)

    def configure_optimizers(self):
        print("Configuring optimizer for ChildModule1")
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return optimizer, scheduler

    def forward(self, x):
        return self.dummy_layer(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        loss = torch.nn.functional.mse_loss(self(x), y)
        return loss

# Define the second child module
class ChildModule2(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.dummy_layer = torch.nn.Linear(1, 1)
        
    
    def configure_optimizers(self):
        print("Configuring optimizer for ChildModule2")
        optimizer = torch.optim.SGD(self.parameters(), lr=1e-2)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2)
        return optimizer, scheduler

    def forward(self, x):
        return self.dummy_layer(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        loss = torch.nn.functional.mse_loss(self(x), y)
        return loss

# Define the parent module that contains the child modules
class ParentModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.child1 = ChildModule1()
        self.child2 = ChildModule2()

    def configure_optimizers(self):
        print("parent's")
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [scheduler]

    def forward(self, x):
        x = self.child1(x)
        x = self.child2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = self.forward(x)
        loss = torch.nn.functional.mse_loss(x, y)
        return loss

# Create the dataset and dataloader
dataset = DummyDataset()
dataloader = DataLoader(dataset, batch_size=32)

# Initialize the parent module
parent_module = ParentModule()

# Initialize the trainer
trainer = pl.Trainer(max_epochs=1)

# Train the model
trainer.fit(parent_module, dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



  | Name   | Type         | Params
----------------------------------------
0 | child1 | ChildModule1 | 2     
1 | child2 | ChildModule2 | 2     
----------------------------------------
4         Trainable params
0         Non-trainable params
4         Total params
0.000     Total estimated model params size (MB)


parent's


c:\Users\proshian\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
c:\Users\proshian\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


# Так как собщений "Configuring optimizer for ChildModule1" и "Configuring optimizer for ChildModule2" не было, их оптимизаторы не создаются